In [11]:
import pandas as pd
import numpy as np
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
import torch
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.metrics import roc_auc_score
from clockwork_helperfunc import *
from clockwork_helperfunc import evaluation
import clockwork_helperfunc 
from imp import reload  
from tqdm import tqdm
reload(clockwork_helperfunc)
import time

#configuration
batch_size = 5
num_epochs = 50
number_units = 7

### Data

In [2]:
training = pd.read_pickle('/Users/leilei/Documents/DS1005/CW/truncate_train_data.p')
val = pd.read_pickle('/Users/leilei/Documents/DS1005/CW/truncate_valid_data.p')

In [3]:
patient_feature_train, label_train, label_time_train = downsample(training, proportional = 2)

In [4]:
sum(label_time_train), sum(label_train*label_time_train)

(529367, 232732)

In [5]:
patient_feature_val = val[0] #before cleaning, original data
label_val = val[1]
label_time_val = val[2]

In [6]:
Counter(label_train), Counter(label_val)

(Counter({0: 3535, 1: 1790}), Counter({0: 4089, 1: 511}))

In [7]:
len(patient_feature_train)

5325

In [8]:
set(label_train[:3535])

{0}

### Model

In [9]:
### Model
#forward
#cell_class, step
class LSTM_Model(nn.Module):
    def __init__(self, number_units, batch_size, mean = 0, std = 0.1, input_dim = 40):
        super(LSTM_Model, self).__init__()
        '''
        scale: the updating frequency, a list. [1,2,4,8,16,32]
        batch_size: the size of batch
        group_size: the number of nodes in each scale, default is 1.
        mean: the mean of Gaussian distribution for initialize weights for hidden layer
        std: the standard devation of the Gaussian distribution for initialize weights for hidden layer
        input_dim: the feature dimension of each time step
        '''
        self.hidden_dim = number_units
        self.input_dim = input_dim
        self.batch_size = batch_size

        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim)
        self.hidden2tag = nn.Linear(self.hidden_dim, 2)
        
    def forward(self, sequence, hidden):#depends on what passed for model.train(), to be filled)
        '''
        sequence: batch  x timestep x number_feature matrix
        hidden: should be h0
        ''' 
        sequence = sequence.transpose(0,1) #now sequence, batch, num_feature
        #print(sequence.size())
        #print(hidden[0].size())
        lstm_out, hidden = self.lstm(sequence, hidden) #hidden is the last one of sequence
        #print('lstm_size')
        #print(lstm_out.size())
        #print(hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sequence)*self.batch_size, -1))
        #previously last step
        #self.hidden2tag(hidden[0])
        #final_score = F.log_softmax(self.hidden2tag(hidden[0]))
        #print(final_score[0])
        tag_scores = F.log_softmax(tag_space)       
        return tag_scores#, final_score[0]#it is batch * class, the first dimension is 1, so remove it

    def init_hidden(self):
        return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
                Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))


In [12]:
lstm = LSTM_Model(number_units, batch_size)
loss = torch.nn.CrossEntropyLoss(ignore_index=-1)  
optimizer = torch.optim.Adam(lstm.parameters(), lr=0.005)
accuracy_list = []
train_loader, validation_loader = reload_data(batch_size,patient_feature_train, label_train, label_time_train,patient_feature_val, label_val, label_time_val)
start = time.time()
for epoch in tqdm(range(num_epochs)):
    for step, (data, label,label_list) in enumerate(train_loader):        
        data, label = Variable(data), Variable(label)
        lstm.zero_grad()
        hidden= lstm.init_hidden()
        output = lstm(data, hidden)
        
        label = label.transpose(0,1).contiguous().view(-1) 
        #output = torch.stack(output, dim=1).view(-1, 2) 
        #print(output.size())
        #print(label.size())
        lossy = loss(output, label)
        lossy.backward()
        optimizer.step()
        #print('one_step')
        if step % 100 ==0:
            print("Epoch: {}; Loss: {}".format(epoch, lossy.data[0]))
                #print('accuracy_on_training: {}'.format(evaluation(train_loader))) 
            acc0, acc1, val_acc, auc = evaluation(validation_loader, lstm,'lstm')
            print('accuracy_on_validation: {}, the acc for live is {}, the acc for dead is {}'.format(val_acc, acc0, acc1)) 
            print('auc ' +str(auc))
    accuracy_list.append(val_acc)
    if ((epoch > 5) and ((accuracy_list[-1] < (accuracy_list[-2] - 0.01)) or (accuracy_list[-1] < (accuracy_list[-3] - 0.01)))):
        print("early stop, accuracy = ", accuracy_list[-2])
        break     
end = time.time()

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 0; Loss: 0.6911388635635376
accuracy_on_validation: 0.12739130434782608, the acc for live is 0.020298361457569088, the acc for dead is 0.9843444227005871
auc 0.502321392079
Epoch: 0; Loss: 0.8523029685020447
accuracy_on_validation: 0.8132608695652174, the acc for live is 0.8941061384201516, the acc for dead is 0.16634050880626222
auc 0.530223323613
Epoch: 0; Loss: 0.22249732911586761
accuracy_on_validation: 0.8002173913043479, the acc for live is 0.879921741257031, the acc for dead is 0.162426614481409
auc 0.521174177869
Epoch: 0; Loss: 0.3109551966190338
accuracy_on_validation: 0.7971739130434783, the acc for live is 0.8728295426754707, the acc for dead is 0.1917808219178082
auc 0.532305182297
Epoch: 0; Loss: 1.2397196292877197
accuracy_on_validation: 0.7984782608695652, the acc for live is 0.8740523355343605, the acc for dead is 0.19373776908023482
auc 0.533895052307
Epoch: 0; Loss: 0.5608583092689514
accuracy_on_validation: 0.7871739130434783, the acc for live is 0.8576669112

  2%|▏         | 1/50 [24:18<19:50:45, 1458.07s/it]

Epoch: 1; Loss: 0.41577595472335815
accuracy_on_validation: 0.7969565217391305, the acc for live is 0.8694057226705796, the acc for dead is 0.2172211350293542
auc 0.54331342885
Epoch: 1; Loss: 0.33770522475242615
accuracy_on_validation: 0.82, the acc for live is 0.8977745169968208, the acc for dead is 0.19765166340508805
auc 0.547713090201
Epoch: 1; Loss: 0.32456016540527344
accuracy_on_validation: 0.7945652173913044, the acc for live is 0.863536316947909, the acc for dead is 0.24266144814090018
auc 0.553098882544
Epoch: 1; Loss: 0.7419033646583557
accuracy_on_validation: 0.802608695652174, the acc for live is 0.8760088041085841, the acc for dead is 0.21526418786692758
auc 0.545636495988
Epoch: 1; Loss: 1.1503275632858276
accuracy_on_validation: 0.8267391304347826, the acc for live is 0.9075568598679383, the acc for dead is 0.18003913894324852
auc 0.543797999406
Epoch: 1; Loss: 0.23464244604110718
accuracy_on_validation: 0.8008695652173913, the acc for live is 0.8733186598190267, the a

  4%|▍         | 2/50 [46:56<18:46:26, 1408.05s/it]

...

Epoch: 9; Loss: 0.4771129786968231
accuracy_on_validation: 0.8039130434782609, the acc for live is 0.8811445341159208, the acc for dead is 0.18590998043052837
auc 0.533527257273
Epoch: 9; Loss: 0.22543716430664062
accuracy_on_validation: 0.8052173913043478, the acc for live is 0.8816336512594767, the acc for dead is 0.19373776908023482
auc 0.53768571017
Epoch: 9; Loss: 0.18438740074634552
accuracy_on_validation: 0.8132608695652174, the acc for live is 0.8901932012717045, the acc for dead is 0.19765166340508805
auc 0.543922432338
Epoch: 9; Loss: 0.505460798740387
accuracy_on_validation: 0.8071739130434783, the acc for live is 0.8840792369772561, the acc for dead is 0.1917808219178082
auc 0.537930029448
Epoch: 9; Loss: 0.4497407376766205
accuracy_on_validation: 0.7954347826086956, the acc for live is 0.863536316947909, the acc for dead is 0.25048923679060664
auc 0.557012776869
Epoch: 9; Loss: 0.4977976083755493
accuracy_on_validation: 0.8093478260869565, the acc for live is 0.88652482269

 20%|██        | 10/50 [2:31:11<10:04:46, 907.17s/it]

Epoch: 10; Loss: 0.28645798563957214
accuracy_on_validation: 0.7845652173913044, the acc for live is 0.8522866226461238, the acc for dead is 0.24266144814090018
auc 0.547474035394
Epoch: 10; Loss: 1.1263988018035889
accuracy_on_validation: 0.7952173913043479, the acc for live is 0.8642699926632429, the acc for dead is 0.24266144814090018
auc 0.553465720402
Epoch: 10; Loss: 0.2589424252510071
accuracy_on_validation: 0.7984782608695652, the acc for live is 0.8647591098067987, the acc for dead is 0.26810176125244617
auc 0.56643043553
Epoch: 10; Loss: 0.5723109245300293
accuracy_on_validation: 0.8023913043478261, the acc for live is 0.8735632183908046, the acc for dead is 0.2328767123287671
auc 0.55321996536
Epoch: 10; Loss: 0.4344883859157562
accuracy_on_validation: 0.7984782608695652, the acc for live is 0.8747860112496944, the acc for dead is 0.18786692759295498
auc 0.531326469421
Epoch: 10; Loss: 0.3905991017818451
accuracy_on_validation: 0.8067391304347826, the acc for live is 0.88016

 22%|██▏       | 11/50 [2:43:52<9:41:00, 893.86s/it] 

Epoch: 11; Loss: 0.32105326652526855
accuracy_on_validation: 0.797608695652174, the acc for live is 0.8662264612374664, the acc for dead is 0.24853228962818003
auc 0.557379375433
Epoch: 11; Loss: 0.5683475732803345
accuracy_on_validation: 0.7813043478260869, the acc for live is 0.8456835412081194, the acc for dead is 0.26614481409001955
auc 0.555914177649
Epoch: 11; Loss: 0.5657588839530945
accuracy_on_validation: 0.7556521739130435, the acc for live is 0.8190266568843237, the acc for dead is 0.24853228962818003
auc 0.533779473256
Epoch: 11; Loss: 0.4094468057155609
accuracy_on_validation: 0.8067391304347826, the acc for live is 0.8779652726828076, the acc for dead is 0.23679060665362034
auc 0.557377939668
Epoch: 11; Loss: 0.2960357964038849
accuracy_on_validation: 0.806304347826087, the acc for live is 0.8789435069699193, the acc for dead is 0.22504892367906065
auc 0.551996215324
Epoch: 11; Loss: 0.2679298222064972
accuracy_on_validation: 0.8021739130434783, the acc for live is 0.8742

In [13]:
print(end - start)

10593.130692958832


In [18]:
### Model
#forward
#cell_class, step
class GRU_Model(nn.Module):
    def __init__(self, number_units, batch_size, mean = 0, std = 0.1, input_dim = 40):
        super(GRU_Model, self).__init__()
        '''
        scale: the updating frequency, a list. [1,2,4,8,16,32]
        batch_size: the size of batch
        group_size: the number of nodes in each scale, default is 1.
        mean: the mean of Gaussian distribution for initialize weights for hidden layer
        std: the standard devation of the Gaussian distribution for initialize weights for hidden layer
        input_dim: the feature dimension of each time step
        '''
        self.hidden_dim = number_units
        self.input_dim = input_dim
        self.batch_size = batch_size

        self.gru = nn.GRU(self.input_dim, self.hidden_dim)
        self.hidden2tag = nn.Linear(self.hidden_dim, 2)
        
    def forward(self, sequence, hidden):#depends on what passed for model.train(), to be filled)
        '''
        sequence: batch  x timestep x number_feature matrix
        hidden: should be h0
        ''' 
        sequence = sequence.transpose(0,1)
        gru_out, hidden = self.gru(sequence, hidden)
        tag_space = self.hidden2tag(gru_out.view(len(sequence)*self.batch_size, -1))
        #self.hidden2tag(hidden[0])
        #final_score = F.log_softmax(self.hidden2tag(hidden[0]))
        tag_scores = F.log_softmax(tag_space)       
        return tag_scores#, final_score[0]
    
    def init_hidden(self):
        return Variable(torch.zeros(1, self.batch_size, self.hidden_dim))

In [ ]:
gru = GRU_Model(number_units, batch_size)
loss = torch.nn.CrossEntropyLoss(ignore_index=-1)  
optimizer = torch.optim.Adam(gru.parameters(), lr=0.05)
accuracy_list = []
train_loader, validation_loader = reload_data(batch_size,patient_feature_train, label_train, label_time_train,patient_feature_val, label_val, label_time_val)


for epoch in range(num_epochs):
    for step, (data, label,label_time) in enumerate(train_loader):        
        data, label = Variable(data), Variable(label)
        gru.zero_grad()
        hidden= gru.init_hidden()
        output = gru(data, hidden)
        
        label = label.transpose(0,1).contiguous().view(-1) 
        lossy = loss(output, label)
        lossy.backward()
        optimizer.step()
        #print('one_step')
        
        if step % 100 ==0:
            print("Epoch: {}; Loss: {}".format(epoch, lossy.data[0]))
                #print('accuracy_on_training: {}'.format(evaluation(train_loader))) 
            acc0, acc1, val_acc, auc = evaluation(validation_loader, lstm,'lstm')
            print('accuracy_on_validation: {}, the acc for live is {}, the acc for dead is {}'.format(val_acc, acc0, acc1)) 
            print('auc ' +str(auc))
    accuracy_list.append(val_acc)
    if ((epoch > 5) and ((accuracy_list[-1] < (accuracy_list[-2] - 0.01)) or (accuracy_list[-1] < (accuracy_list[-3] - 0.01)))):
        print("early stop, accuracy = ", accuracy_list[-2])
        break         

Epoch: 0; Loss: 0.5425668358802795
accuracy_on_validation: 0.7941304347826087, the acc for live is 0.8672046955245781, the acc for dead is 0.20939334637964774
auc 0.538299020952
Epoch: 0; Loss: 1.5264885425567627
accuracy_on_validation: 0.8015217391304348, the acc for live is 0.8708730741012473, the acc for dead is 0.2465753424657534
auc 0.558724208284
Epoch: 0; Loss: 0.6992398500442505
accuracy_on_validation: 0.8126086956521739, the acc for live is 0.879921741257031, the acc for dead is 0.273972602739726
auc 0.576947171998
Epoch: 0; Loss: 0.3505977988243103
accuracy_on_validation: 0.7969565217391305, the acc for live is 0.8628026412325752, the acc for dead is 0.2700587084148728
auc 0.566430674824
Epoch: 0; Loss: 0.6616052389144897
accuracy_on_validation: 0.7978260869565217, the acc for live is 0.8681829298116899, the acc for dead is 0.23483365949119372
auc 0.551508294651
Epoch: 0; Loss: 0.7396223545074463
accuracy_on_validation: 0.797608695652174, the acc for live is 0.865492785522132